In [14]:
import csv
import glob
import numpy as np
import pandas as pd
from numpy import math

In [92]:
input_dir = 'C:/Users/Nayeem/Documents/Python Projects/Jupyter Notebook/My Project Work/Raw Data/Person 5/Pocket/'

## List of the name of all of the file in the directory

In [93]:
file_list = glob.glob("{}{}".format(input_dir, '*csv'))
file_list

['C:/Users/Nayeem/Documents/Python Projects/Jupyter Notebook/My Project Work/Raw Data/Person 5/Pocket\\Project-measurements_510_pocket_2020-03-09_17-16-28_formatted.csv',
 'C:/Users/Nayeem/Documents/Python Projects/Jupyter Notebook/My Project Work/Raw Data/Person 5/Pocket\\Project-measurements_51_pocket_2020-03-09_17-09-43_formatted.csv',
 'C:/Users/Nayeem/Documents/Python Projects/Jupyter Notebook/My Project Work/Raw Data/Person 5/Pocket\\Project-measurements_52_pocket_2020-03-09_17-11-02_formatted.csv',
 'C:/Users/Nayeem/Documents/Python Projects/Jupyter Notebook/My Project Work/Raw Data/Person 5/Pocket\\Project-measurements_53_pocket_2020-03-09_17-11-48_formatted.csv',
 'C:/Users/Nayeem/Documents/Python Projects/Jupyter Notebook/My Project Work/Raw Data/Person 5/Pocket\\Project-measurements_54_pocket_2020-03-09_17-12-29_formatted.csv',
 'C:/Users/Nayeem/Documents/Python Projects/Jupyter Notebook/My Project Work/Raw Data/Person 5/Pocket\\Project-measurements_55_pocket_2020-03-09_17-1

In [94]:
def getHeaderIndex(header_index, symbol, tmp_path):
        with open(tmp_path, newline='', encoding='utf-8') as inf:
            reader = csv.reader(inf)
            for index, row in enumerate(reader):
                if symbol in row[0]:
                    header_index = index
        return header_index

### We can't drop NaN with standard pandas library function. so this is an alternative solution

In [95]:
def dropNaN(df):
    '''
    :param df: the dataframe
    :return: the filtered dataframe without NaN values.
    '''
    data = []

    for index, row in enumerate(df.values.tolist()):
           entries = []
           for entry in row:
               if isinstance(entry, float) and math.isnan(entry):
                   entry = 0
               entries.append(entry)
           data.append(entries)

    df_filtered = pd.DataFrame(data=data, columns =df.columns.values)

    return df_filtered

### Taking care of headers or comments on dataset

In [96]:
def build_dataframe(header_index):
    '''
    :param header_index: get the header index from file
    :return: the dataframe from .csv file
    '''
    data = None

    # If there is no comment in the .csv file, so we do not need to skip the useless rows.
    if (header_index == 0):
        data = pd.read_csv(file, encoding='utf-8-sig')
        
    # If there are comments in the .csv file, we have to read the first line after header_index
    elif (header_index != 0):
        data = pd.read_csv(file, skiprows=header_index, encoding='utf-8-sig')

    header = data.columns.values
    df = pd.DataFrame(data=data, columns=header)
    return df
    df.head()

symbol = '#'

In [97]:
def sublists(length, block_len, steps):
    def chunks(l):
        """Yield successive n-sized chunks from l"""
        for i in range(1, len(l), steps):
            yield l[i:i + block_len]

    blocks = list(chunks(range(0, length)))

    return blocks

In [98]:
def buil_columns_name(header_name, name):
    header_new_name = []
    header_new_name.append(header_name[0])
    header_new_name.append(header_name[1])
    for names in header_name[2:]:
        header_new_name.append('{}{}'.format(names, name))
    return header_new_name

### Sliding window and feature extraction

In [99]:
def overlapping(df_data):

    frequency = 50
    step = int(frequency/2)

    # index of label header in dataframe
    label_index = 1

    dyn_index = 2
    header_list = df_data.columns.values

    label_header = header_list[label_index]
    dyn_header = header_list[dyn_index:len(header_list)]
    #dyn_header = header_list[dyn_index:len(header_list)-1]

    blocks = sublists(length=len(df_data), block_len=frequency, steps=step)

    data1 = []
    data2 = []
    data3 = []
    data4 = []
    data5 = []

    for block in blocks:
        block_informations1 = []
        block_informations2 = []
        block_informations3 = []
        block_informations4 = []
        block_informations5 = []

        header_list1 = buil_columns_name(header_list, 'mean')
        header_list2 = buil_columns_name(header_list, 'var')
        header_list3 = buil_columns_name(header_list, 'rg')
        header_list4 = buil_columns_name(header_list, 'min')
        header_list5 = buil_columns_name(header_list, 'max')

         # select the sub dataframe
        start = block[0]
        end = block[len(block)-1]
        my_df = df_data.loc[start:end]

        # dataframe operations
        label_list = list(my_df[label_header].values)
        label_name = max(label_list,key=label_list.count)

        block_informations1.append(label_name)
        block_informations2.append(label_name)
        block_informations3.append(label_name)
        block_informations4.append(label_name)
        block_informations5.append(label_name)

         # Calculating mean, variance, range, min, max
        for col in dyn_header:
            mean = my_df[col].mean()
            block_informations1.append(mean)

            var = my_df[col].var()
            block_informations2.append(var)

            ranges = (max(my_df[col]) - min(my_df[col]))
            block_informations3.append(ranges)

            minimums = min(my_df[col])
            block_informations4.append(minimums)

            maximums = max(my_df[col])
            block_informations5.append(maximums)

        data1.append(block_informations1)
        data2.append(block_informations2)
        data3.append(block_informations3)
        data4.append(block_informations4)
        data5.append(block_informations5)

    df1 = pd.DataFrame(data=data1, columns=header_list1[label_index:len(header_list) ])
    print(df1)
    df2 = pd.DataFrame(data=data2, columns=header_list2[label_index:len(header_list) ])
    df2 = df2.drop('label',axis=1)

    df3 = pd.DataFrame(data=data3, columns=header_list3[label_index:len(header_list)])
    df3 = df3.drop('label',axis=1)

    df4 = pd.DataFrame(data=data4, columns=header_list4[label_index:len(header_list)])
    df4 = df4.drop('label', axis=1)

    df5 = pd.DataFrame(data=data5, columns=header_list5[label_index:len(header_list)])
    df5 = df5.drop('label', axis=1)
    print(df1)
    df = pd.concat([df1, df2, df3, df4, df5],axis=1)

    return df

filecounter = 1
for file in file_list:

        print('... processing ', file)

        header_index = 0

        # get the header informations from .csv file
        header_index = getHeaderIndex(header_index, symbol, file)

        # build dataframe
        df = build_dataframe(header_index)

        # drop all rows including NaN values
        #df_data = dropNaN(df)
        df_data = df.fillna(0)

        # create overlapping dataframe with windows
        df_overlap = overlapping(df_data)

        # write .csv file
        path = "{}new_{}{}".format(input_dir,filecounter,'.csv')
        df_overlap.to_csv(path, index=False)

        filecounter+=1


... processing  C:/Users/Nayeem/Documents/Python Projects/Jupyter Notebook/My Project Work/Raw Data/Person 5/Pocket\Project-measurements_510_pocket_2020-03-09_17-16-28_formatted.csv
             label  TYPE_GYROSCOPE-Xmean  TYPE_GYROSCOPE-Ymean  \
0            Stand             -0.066055              0.419200   
1            Stand             -1.123133             -1.191729   
2            Stand             -0.117315             -0.404973   
3            Stand              0.702296             -0.565615   
4            Stand              0.423453             -0.905217   
5            Stand             -0.048241              0.184990   
6            Stand             -0.043574              0.090671   
7            Stand              0.013882              0.034254   
8            Stand              0.123983              0.038642   
9            Stand              0.131631             -0.036408   
10           Stand             -0.020374             -0.103751   
11           Stand        

     label  TYPE_ACCELEROMETER-Xmean  TYPE_ACCELEROMETER-Ymean  \
0    Stand                 -2.111870                  2.738001   
1    Stand                 -2.369485                  1.159651   
2    Stand                 -2.338265                  0.914484   
3    Stand                 -2.019454                  1.845251   
4    Stand                 -1.947053                  1.967931   
5    Stand                 -1.889975                  1.697098   
6    Stand                 -1.109563                  1.865267   
7    Stand                 -0.280408                  2.566577   
8    Stand                 -0.163762                  2.903585   
9    Stand                 -0.181098                  2.973208   
10   Stand                  0.964569                  2.978858   
11   Stand                  3.087554                  2.951565   
12   Stand                  4.866063                  2.888454   
13   Stand                  7.415697                  1.668658   
14   Stand

             label  TYPE_GYROSCOPE-Xmean  TYPE_GYROSCOPE-Ymean  \
0            Stand              1.097165             -1.075210   
1            Stand              0.490074             -0.402522   
2            Stand              0.177442             -0.415305   
3            Stand              0.693218             -2.668503   
4            Stand              1.770113             -4.347785   
5            Stand              2.304875             -2.281178   
6            Stand              2.291880             -0.328314   
7            Stand              0.933503             -0.644950   
8            Stand             -1.229892             -0.617871   
9            Stand             -1.208183              1.053159   
10           Stand             -0.107216              0.376650   
11           Stand              0.236712             -0.838621   
12           Stand              0.016396              0.174616   
13           Stand             -0.286858              0.069175   
14        

             label  TYPE_ACCELEROMETER-Xmean  TYPE_ACCELEROMETER-Ymean  \
0            Stand                  8.241025                 -5.140249   
1            Stand                  8.107620                 -4.764167   
2            Stand                  7.508592                 -5.073976   
3            Stand                  3.202864                 -8.371461   
4            Stand                 -1.527689                -10.639728   
5            Stand                 -1.352818                -10.239801   
6            Stand                  0.479794                 -9.541939   
7            Stand                  1.227645                 -9.710490   
8            Stand                  1.876184                 -9.818517   
9            Stand                  1.797177                 -9.803769   
10           Stand                  1.772947                 -9.575266   
11   Start to walk                  1.296119                 -9.638377   
12   Start to walk                  0.

             label  TYPE_GYROSCOPE-Xmean  TYPE_GYROSCOPE-Ymean  \
0            Stand              0.240507             -0.677891   
1            Stand              0.183728             -1.076278   
2            Stand              0.182237              0.068920   
3            Stand              0.133340             -0.187207   
4            Stand              0.018867             -0.065788   
5            Stand              0.063626             -0.077442   
6            Stand              0.181420             -0.192402   
7            Stand              0.125218             -0.237143   
8            Stand              0.035372             -0.153990   
9            Stand              0.054438             -0.055475   
10           Stand              0.055504             -0.026924   
11           Stand              0.062110              0.007456   
12           Stand              0.053972              0.023090   
13           Stand              0.015414             -0.023733   
14        

             label  TYPE_GYROSCOPE-Xmean  TYPE_GYROSCOPE-Ymean  \
0            Stand              0.168794             -0.385629   
1            Stand             -0.489324             -2.125815   
2            Stand             -0.656016             -1.014175   
3            Stand             -1.080540              1.089991   
4            Stand             -0.081861             -0.126643   
5            Stand              0.748976             -0.905966   
6            Stand              0.411757              0.415063   
7            Stand             -0.018267              0.385788   
8            Stand             -0.034202              0.129830   
9            Stand              0.212616             -0.128853   
10           Stand              0.320608             -0.223466   
11           Stand              0.120785             -0.151793   
12           Stand              0.083056             -0.184691   
13           Stand              0.065545              0.000742   
14        

             label  TYPE_GYROSCOPE-Xmean  TYPE_GYROSCOPE-Ymean  \
0            Stand              0.598857             -0.937709   
1            Stand              1.282776             -2.929983   
2            Stand              1.856741             -4.872938   
3            Stand              3.067245             -2.641875   
4            Stand              2.437014              0.760722   
5            Stand              0.669104             -1.640612   
6            Stand             -0.617816             -2.877807   
7            Stand             -1.506413             -0.020178   
8            Stand             -0.577229              0.563691   
9            Stand              0.378756             -0.807577   
10           Stand              0.047138             -0.450583   
11           Stand              0.039997              0.116045   
12           Stand              0.078537              0.237181   
13           Stand              0.129691              0.285098   
14        

             label  TYPE_GYROSCOPE-Xmean  TYPE_GYROSCOPE-Ymean  \
0            Stand             -0.711768             -1.049347   
1            Stand             -0.660830             -0.808473   
2            Stand             -0.721231             -0.639566   
3            Stand             -0.743942             -0.064901   
4            Stand              0.159608             -1.160713   
5            Stand              0.523971             -1.098712   
6            Stand              0.109034              0.048976   
7            Stand             -0.080196              0.175418   
8            Stand             -0.110211              0.150938   
9            Stand             -0.043894              0.147827   
10           Stand              0.024340              0.047038   
11           Stand              0.026065             -0.069047   
12           Stand              0.010065             -0.149069   
13           Stand              0.015432             -0.191487   
14        

             label  TYPE_GYROSCOPE-Xmean  TYPE_GYROSCOPE-Ymean  \
0            Stand              0.995500             -3.324813   
1            Stand              1.909241             -5.698967   
2            Stand              3.641208             -4.211585   
3            Stand              2.418776             -0.003900   
4            Stand             -0.368395             -0.864183   
5            Stand             -1.682883             -1.083478   
6            Stand             -0.273504             -0.014211   
7            Stand              0.807287             -0.632574   
8            Stand              0.149598             -0.623325   
9            Stand              0.072193             -0.153647   
10           Stand              0.166953             -0.088689   
11           Stand              0.111709              0.087498   
12           Stand             -0.002375              0.191782   
13           Stand             -0.109233              0.091055   
14        

             label  TYPE_GYROSCOPE-Xmean  TYPE_GYROSCOPE-Ymean  \
0            Stand             -0.239135             -1.128344   
1            Stand             -0.198507             -2.482230   
2            Stand              1.471035             -4.413576   
3            Stand              3.389464             -3.099109   
4            Stand              1.992820             -0.473574   
5            Stand             -0.254710             -1.068542   
6            Stand             -0.642849             -0.983494   
7            Stand              0.490628             -0.922625   
8            Stand              1.094181             -1.434970   
9            Stand              0.348692             -0.783369   
10           Stand             -0.028344             -0.108787   
11           Stand              0.108219              0.256528   
12           Stand              0.247873              0.289872   
13           Stand              0.148783              0.070668   
14        